# Experiments

### Setup

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [ ]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """Hi Anshul this side,
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [4]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset2"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-eb930d25' at:
https://smith.langchain.com/o/9f7786d4-df30-4628-baf7-21b496a6bd87/datasets/93405f1a-00fd-4490-8705-d45812fe15f4/compare?selectedSessions=eaddf0e1-cdc9-4eb1-9148-edf4278315df




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can I integrate LangSmith with other cloud ser...,"Yes, you can integrate LangSmith with other cl...",None,"Yes, LangSmith can be integrated with various ...",1,4.196352,fad3e423-7558-40e8-8e44-caeda61a3056,b6a398c7-a831-4c76-9699-fdf6a3f5339c
1,What languages does the LangSmith SDK support?,The LangSmith SDK supports Python and JavaScript.,None,The LangSmith SDK currently supports programmi...,1,1.491406,29a11b81-320f-4df7-a653-ecd18de5e3b7,6b8fa135-a9c9-43fb-91c2-e319c0724ff3
2,How does LangSmith handle data privacy?,"LangSmith complies with GDPR, is SOC 2 Type 2 ...",None,LangSmith takes data privacy seriously by impl...,1,3.961935,8acfbb7a-89ce-4a94-aec9-07927e37350c,fd5875e9-4504-495b-9db2-841c7ca19f8c
3,Is there a Javascript Langsmith SDK?,"Yes, there is a JavaScript SDK for LangSmith. ...",None,Yes there is a Javascript Langsmith SDK,0,2.196132,91678c2d-cf8b-438b-951a-250e2a91a851,928f3dc4-a36c-4991-bbdd-5155587b930b
4,In langchain how to tag the dataset,"In LangChain, to tag a dataset, you can create...",None,"To tag a dataset in LangChain, you need to cre...",1,2.818653,fb7ce0ff-ca87-4c82-883f-006477da39bb,28487261-013a-488b-afa1-c64ac78e78b6
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,3.217826,243ea8f7-405d-47b8-9508-4b5d83bd8078,b5773d51-f636-4640-9198-6a3d4c2db25c
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.703184,25bec321-a7e4-4012-8547-12c80dfa87f8,78d68daf-d6ba-4515-8fd7-f8b001df8b83
7,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.784939,3011370d-685c-44c9-b56c-06a3999ea259,890dd9b3-80a3-4e0e-acb3-8b50505213f5
8,Can LangSmith be used for finetuning and model...,"LangSmith is designed for monitoring, evaluati...",None,"Yes, LangSmith can be used for fine-tuning and...",1,2.234213,40fcbaf6-a341-4860-b5d7-f6f7b7486838,34b14435-c270-4cc1-b1d3-100cd14ef262
9,What testing capabilities does LangSmith have?,"LangSmith offers integration with LangChain, p...",None,LangSmith offers capabilities for creating dat...,1,1.996782,6d471448-e380-4fb7-8bf9-3c888ddabdeb,7c717302-08cb-430e-8987-16a2e4f15501


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [6]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-3676ebbf' at:
https://smith.langchain.com/o/9f7786d4-df30-4628-baf7-21b496a6bd87/datasets/93405f1a-00fd-4490-8705-d45812fe15f4/compare?selectedSessions=6157df45-adf6-47c2-a84a-c114a6cda53c




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can I integrate LangSmith with other cloud ser...,LangSmith can be integrated with other service...,None,"Yes, LangSmith can be integrated with various ...",1,1.976340,fad3e423-7558-40e8-8e44-caeda61a3056,897447ef-994e-457f-b274-7f10252fee93
1,What languages does the LangSmith SDK support?,The LangSmith SDK supports JavaScript and Type...,None,The LangSmith SDK currently supports programmi...,0,1.239411,29a11b81-320f-4df7-a653-ecd18de5e3b7,6a998cee-c800-466a-b771-5e4e6c951300
2,How does LangSmith handle data privacy?,"LangSmith complies with the GDPR, SOC 2 Type 2...",None,LangSmith takes data privacy seriously by impl...,1,1.648876,8acfbb7a-89ce-4a94-aec9-07927e37350c,ab39a06f-d749-4b0d-9995-1d6cffbea5b8
3,Is there a Javascript Langsmith SDK?,"Yes, there is a JS/TS SDK available for LangSm...",None,Yes there is a Javascript Langsmith SDK,0,1.671207,91678c2d-cf8b-438b-951a-250e2a91a851,626e6e47-376a-41b1-a111-963ffabc8008
4,In langchain how to tag the dataset,"To tag a dataset in LangChain, you can follow ...",None,"To tag a dataset in LangChain, you need to cre...",1,1.600756,fb7ce0ff-ca87-4c82-883f-006477da39bb,0c786e82-216b-43c6-a32d-c8881bc6b2e4
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.609378,243ea8f7-405d-47b8-9508-4b5d83bd8078,17ffc77e-1b76-458a-8e6e-af355ae796e0
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",1,2.112678,25bec321-a7e4-4012-8547-12c80dfa87f8,62801ab2-7a52-4f86-9d06-1aab6b1271b0
7,Does LangSmith support online evaluation?,"Yes, LangSmith does support online evaluation ...",None,"Yes, LangSmith supports online evaluation as a...",0,1.898604,3011370d-685c-44c9-b56c-06a3999ea259,3d86909f-5e66-4457-94a3-5b7758a67dfd
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.486551,40fcbaf6-a341-4860-b5d7-f6f7b7486838,3793f8d4-a1ad-48b5-a282-334b9f1b5303
9,What testing capabilities does LangSmith have?,LangSmith offers a range of testing capabiliti...,None,LangSmith offers capabilities for creating dat...,1,2.215532,6d471448-e380-4fb7-8bf9-3c888ddabdeb,4b4d4974-07ce-4dd7-8243-0844da99e38d


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [9]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial"
)

View the evaluation results for experiment: 'Crucial-aca2cd97' at:
https://smith.langchain.com/o/9f7786d4-df30-4628-baf7-21b496a6bd87/datasets/93405f1a-00fd-4490-8705-d45812fe15f4/compare?selectedSessions=cbe53fbc-c084-44fb-8663-1a419785b865




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.266094,243ea8f7-405d-47b8-9508-4b5d83bd8078,24d2952a-bef1-48e3-99e0-23a9cbd057dd
1,What languages does the LangSmith SDK support?,The LangSmith SDK supports Python and JavaScri...,None,The LangSmith SDK currently supports programmi...,1,1.103635,29a11b81-320f-4df7-a653-ecd18de5e3b7,9987c5a5-78e1-43e0-8b1a-e92033829cbf
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.899567,40fcbaf6-a341-4860-b5d7-f6f7b7486838,85ee63e3-2ba4-449e-b15e-f8eac5f1b5be
3,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,2.640903,87162450-1766-44a3-a214-fac44bb6435e,549bbdac-6250-45ca-b2eb-c9112021563e
4,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,2.322242,9f8cecc6-d72d-4580-9ed8-6db971087300,f6e63ccf-1aa6-40ab-98f4-b41ae69d1084


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [ ]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            
            "3424242-2sfewf-2sdfsd",
            "8acfbb7a-89ce-4a94-aec9-07927e37350c"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-d611d8f7' at:
https://smith.langchain.com/o/9f7786d4-df30-4628-baf7-21b496a6bd87/datasets/93405f1a-00fd-4490-8705-d45812fe15f4/compare?selectedSessions=266a5cfd-d463-475c-8880-69d7a274f168




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What languages does the LangSmith SDK support?,The LangSmith SDK supports Python and JavaScri...,None,The LangSmith SDK currently supports programmi...,1,1.705018,29a11b81-320f-4df7-a653-ecd18de5e3b7,29127cdc-9269-4d70-ad1f-7b3b8e7dc2ca
1,How does LangSmith handle data privacy?,"LangSmith complies with GDPR, SOC 2 Type 2, an...",None,LangSmith takes data privacy seriously by impl...,1,1.444008,8acfbb7a-89ce-4a94-aec9-07927e37350c,9b2abea8-72ed-43be-bd0d-eaa973086e5d


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [12]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-2b3b14bf' at:
https://smith.langchain.com/o/9f7786d4-df30-4628-baf7-21b496a6bd87/datasets/93405f1a-00fd-4490-8705-d45812fe15f4/compare?selectedSessions=42f4ce3a-95d1-4d18-a1cf-4f25438d94b9




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.105100,243ea8f7-405d-47b8-9508-4b5d83bd8078,5a9d572d-17cb-4b0b-ae47-b4e362650461
1,What languages does the LangSmith SDK support?,The LangSmith SDK supports JavaScript (JS) and...,None,The LangSmith SDK currently supports programmi...,0,1.403960,29a11b81-320f-4df7-a653-ecd18de5e3b7,e12752f9-0d5b-4268-ad09-019e5bbdeb34
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.326447,40fcbaf6-a341-4860-b5d7-f6f7b7486838,3d119cfa-2301-40a4-8261-80e39abccf1f
3,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.513942,87162450-1766-44a3-a214-fac44bb6435e,1b2c564b-7813-4ac3-a20f-d8ed70a22d0c
4,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,1.698726,9f8cecc6-d72d-4580-9ed8-6db971087300,ffa63616-dd7d-4b82-a6ab-9a2c5e641ef4
5,Can I integrate LangSmith with other cloud ser...,LangSmith can be integrated with an external C...,None,"Yes, LangSmith can be integrated with various ...",1,1.615527,fad3e423-7558-40e8-8e44-caeda61a3056,4b15a836-8a83-419a-9c20-53c59d713d9e
6,How does LangSmith handle data privacy?,"LangSmith complies with the GDPR, SOC 2 Type 2...",None,LangSmith takes data privacy seriously by impl...,1,1.636850,8acfbb7a-89ce-4a94-aec9-07927e37350c,87baa22e-73b8-4ff6-a9c3-0ba5fbe5029e
7,Is there a Javascript Langsmith SDK?,"Yes, there is a JavaScript LangSmith SDK avail...",None,Yes there is a Javascript Langsmith SDK,0,1.119274,91678c2d-cf8b-438b-951a-250e2a91a851,9df7b7f6-04ba-4820-a7de-99321979d40f
8,In langchain how to tag the dataset,"In LangChain, you can tag a dataset by creatin...",None,"To tag a dataset in LangChain, you need to cre...",1,1.651623,fb7ce0ff-ca87-4c82-883f-006477da39bb,510dc8c2-193e-48cd-8fa8-a7e3688eb969
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.248659,25bec321-a7e4-4012-8547-12c80dfa87f8,67e7881b-b6c8-4c8a-865c-d4080eb4bbb2


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [11]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-6a98dfa6' at:
https://smith.langchain.com/o/9f7786d4-df30-4628-baf7-21b496a6bd87/datasets/93405f1a-00fd-4490-8705-d45812fe15f4/compare?selectedSessions=cfd4488e-aeee-4284-931e-d3336fefc5ec




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What languages does the LangSmith SDK support?,The LangSmith SDK supports Python and JavaScri...,None,The LangSmith SDK currently supports programmi...,1,2.054951,29a11b81-320f-4df7-a653-ecd18de5e3b7,349d581c-d76e-423d-893f-98ff780324e0
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.154476,243ea8f7-405d-47b8-9508-4b5d83bd8078,18beb070-3782-4917-915c-25d5862c205e
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on observabilit...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.241650,40fcbaf6-a341-4860-b5d7-f6f7b7486838,96dab83e-7b57-492a-a577-4fdf2f9e7f11
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.581414,9f8cecc6-d72d-4580-9ed8-6db971087300,6db0b999-8f04-4f58-bc01-ef82a0d24e36
4,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.812481,87162450-1766-44a3-a214-fac44bb6435e,54098f7b-4c11-4424-b25f-9b91b7a2699d
5,Can I integrate LangSmith with other cloud ser...,LangSmith can be integrated with an external C...,None,"Yes, LangSmith can be integrated with various ...",1,1.923617,fad3e423-7558-40e8-8e44-caeda61a3056,07236ab4-0009-4c95-b1db-d97d6f62456e
6,Is there a Javascript Langsmith SDK?,"Yes, LangSmith does offer a JavaScript (JS/TS)...",None,Yes there is a Javascript Langsmith SDK,0,1.488158,91678c2d-cf8b-438b-951a-250e2a91a851,f4719f46-3197-4027-b7a9-49855169ef8b
7,In langchain how to tag the dataset,"In LangChain, to tag a dataset, you can create...",None,"To tag a dataset in LangChain, you need to cre...",1,1.676864,fb7ce0ff-ca87-4c82-883f-006477da39bb,6ea025f0-1806-460f-ad41-1b6409c71f99
8,How does LangSmith handle data privacy?,"LangSmith complies with GDPR, is SOC 2 Type 2 ...",None,LangSmith takes data privacy seriously by impl...,1,2.148028,8acfbb7a-89ce-4a94-aec9-07927e37350c,f7643d83-6dd5-4cdc-9233-259ab4451e8c
9,What testing capabilities does LangSmith have?,LangSmith has off-the-shelf evaluators for dif...,None,LangSmith offers capabilities for creating dat...,1,1.946758,6d471448-e380-4fb7-8bf9-3c888ddabdeb,8550ca67-5a6e-47c0-8754-d3709ac60c2b


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [13]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-3a7be64f' at:
https://smith.langchain.com/o/9f7786d4-df30-4628-baf7-21b496a6bd87/datasets/93405f1a-00fd-4490-8705-d45812fe15f4/compare?selectedSessions=aef7bdb8-17aa-45bf-91b2-d53af431c6e9




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.697141,243ea8f7-405d-47b8-9508-4b5d83bd8078,a69132b1-76f0-47a9-aaff-30737eb55097
1,What languages does the LangSmith SDK support?,LangSmith SDK supports JavaScript and TypeScri...,None,The LangSmith SDK currently supports programmi...,0,1.491611,29a11b81-320f-4df7-a653-ecd18de5e3b7,5f7bbbf6-bd7b-45b4-b0df-235a450e8671
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for observabil...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.896153,40fcbaf6-a341-4860-b5d7-f6f7b7486838,60c33b64-2334-474b-bc82-f9b3fb0fd2c6
3,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.749296,87162450-1766-44a3-a214-fac44bb6435e,0eff820e-4785-4333-90e2-87d054d7ab7b
4,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,2.049255,9f8cecc6-d72d-4580-9ed8-6db971087300,a7db1043-098e-4cf3-b2e3-ab8b43e908dd
5,Can I integrate LangSmith with other cloud ser...,LangSmith can be integrated with an external C...,None,"Yes, LangSmith can be integrated with various ...",0,1.675909,fad3e423-7558-40e8-8e44-caeda61a3056,d5e0cd33-87d6-454e-b259-ad8fe6e057fb
6,How does LangSmith handle data privacy?,"LangSmith complies with the GDPR, SOC 2 Type 2...",None,LangSmith takes data privacy seriously by impl...,1,1.446820,8acfbb7a-89ce-4a94-aec9-07927e37350c,21accd40-79bd-400d-8b42-96834bb5e56d
7,Is there a Javascript Langsmith SDK?,"Yes, there is a JS/TS SDK for LangSmith which ...",None,Yes there is a Javascript Langsmith SDK,0,1.744960,91678c2d-cf8b-438b-951a-250e2a91a851,ff38ea5f-a005-4b65-9514-2ee9be34c5a0
8,In langchain how to tag the dataset,"In LangChain, to tag a dataset, you can use th...",None,"To tag a dataset in LangChain, you need to cre...",1,2.189173,fb7ce0ff-ca87-4c82-883f-006477da39bb,ffcd4a7a-8454-48a5-8936-340f0dbcc5a6
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.442017,25bec321-a7e4-4012-8547-12c80dfa87f8,042fc539-aa64-42c1-8ab2-f6703805db71
